In [ ]:
basePath='D:\\PM2\\IV2A_TD-DNC\\'
dbType='IV2A_Imagery'
random_seed = 32654

Channels=3
features=1000
nb_classes=2
batch_size=512

## setup

In [2]:
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False


import tensorflow as tf
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)


import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


In [ ]:
import os,sys,inspect,glob,warnings

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
warnings.filterwarnings('ignore')


sys.path.insert(0,'D:\\PM2\\')


print('**** ok ****')

In [ ]:
ghjk


#!pip uninstall tensorflow==2.2.0 --yes
pip uninstall keras --yes

      #!pip install argcomplete
pip install keras==2.2.4
#!pip install tensorflow==1.10



pip install ipython-autotime
pip install plot_keras_history

print('~'*50)


%tensorflow_version 1.10
import tensorflow
print(tensorflow.__version__)
print('~'*50)

print('ok')


In [ ]:
# %tensorflow_version 1.10
import tensorflow
print(tensorflow.__version__)

## Prepare Data 25920

In [ ]:
import scipy.io
import pandas as pd
import numpy as np
import pickle as pkl
import gc


def prepare_data():
  #~~~~~~~~~~~~~~~~~~~~~~~~~~~     prepare data   ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  

  with open('D:\ds\MMCNN_2A_data_raw.pkl','rb') as f: #read
    _Data = pkl.load(f)
  
  print(_Data.shape)
  print('-------------------------------------------')


  with open('D:\ds\MMCNN_2A_label_raw.pkl','rb') as f: #read
    _Target = pkl.load(f)
  
  print(_Target.shape)
  print('-------------------------------------------')


  _Data = _Data.reshape( _Data.shape[0] , 1000 * 3 )

  print(' _Data shape is : ' , _Data.shape)
  print(' _Target shape is : ' , _Target.shape)
  

  return _Data, _Target



_Data, _Target= prepare_data()
print(' Load data ==> OK  ')

In [ ]:
from sklearn.model_selection import train_test_split

def split_and_prepare_Data(_Data, _Target):
  X_train, X_test, y_train, y_test = train_test_split( _Data, _Target , test_size=0.3 , random_state=random_seed  , shuffle=True)  #276159



  print("\x1b[31m  ....................................  \x1b[0m")
  print('X_train: ', X_train.shape)
  print('X_test: ', X_test.shape)
  print('y_train: ', y_train.shape)
  print('y_test: ', y_test.shape)
  print("\x1b[31m  ................. Total ...................  \x1b[0m")





  X_train=X_train[:15360]
  y_train=y_train[:15360]

  X_test=X_test[:7168]
  y_test=y_test[:7168]
  


  print()
  print("\x1b[31m  .................. Summary ..................  \x1b[0m")
  print('X_train: ', X_train.shape)
  print('X_test: ', X_test.shape)
  print('y_train: ', y_train.shape)
  print('y_test: ', y_test.shape)
 

  print("\x1b[31m  ....................................  \x1b[0m")



  import pickle as pkl
  with open(basePath+'Imagery_temp_X_test_IV2A.pkl','wb') as f: #write
    pkl.dump( X_test , f)

  with open(basePath +'Imagery_temp_y_test_IV2A.pkl','wb') as f: #write
    pkl.dump( y_test , f)

  del  X_test, y_test


  return X_train,y_train


X_train, y_train= split_and_prepare_Data(_Data, _Target)  



In [ ]:
del _Data, _Target

In [ ]:
print( X_train.shape)
print(y_train.shape)

In [ ]:
y_train2=[np.where(r==1)[0][0] for r in np.array( y_train ) ]

## Reshaper

In [3]:
from sklearn.base import BaseEstimator , TransformerMixin

class CustomLSTMReshaper(BaseEstimator , TransformerMixin):
    
     def fit(self,X,y=None):
        return self    
    
     def transform(self, X, y=None):     
        return X.reshape(( len(X), features, Channels ,1 ))

## Train Metrics

In [4]:
from keras import backend as K


def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


# https://medium.com/@mostafa.m.ayoub/customize-your-keras-metrics-44ac2e2980bd
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred) #precision_m
    recall = recall_m(y_true, y_pred) #recall_m
    return 2*( (precision*recall)/(precision+recall+K.epsilon() )   )


def specificity_m(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1 - y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())


# implement at: https://www.sabinasz.net/unbalanced-classes-machine-learning/
def sensitivity_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())    





Using TensorFlow backend.


## Test Metric

In [5]:

## Metric library
from imblearn.metrics import sensitivity_score ,specificity_score #geometric_mean_score

from sklearn.metrics import(
                            accuracy_score,
                            precision_score, 
                            recall_score,
                            f1_score,                                                      
                            make_scorer,                            
                            cohen_kappa_score
                            )


def make_experiments_for_tuner():
       
    accuracy = make_scorer(accuracy_score)
    precision = make_scorer(precision_score , average = 'macro' )
    recall = make_scorer(recall_score , average = 'macro')
    f1 = make_scorer(f1_score  , average = 'macro')
    sensitivity = make_scorer( sensitivity_score  , average = 'macro')     
    specificity = make_scorer( specificity_score , average = 'macro')
    kappa= make_scorer(cohen_kappa_score   )

    scores_for_grid = {
    "accuracy":accuracy,
    "precision":precision, 
    "recall":recall,
    "f1":f1,
    "sensitivity":sensitivity,
    "specificity":specificity,
    "kappa":kappa
    }
    
    return scores_for_grid


def get_experiments(true_label,pred_label):
    
     return  {
    "test_accuracy":accuracy_score(true_label,pred_label ),
    "test_precision":precision_score(true_label,pred_label , average='macro' ), 
    "test_recall":recall_score(true_label,pred_label , average='macro' ),
    "test_f1":f1_score(true_label,pred_label , average='macro' ),
    "test_specificity":specificity_score(true_label,pred_label , average='macro'),    
    "test_sensitivity":sensitivity_score(true_label,pred_label , average='macro'),
    "test_kappa":cohen_kappa_score(true_label,pred_label   )
     }



## model





In [8]:
# need these for ShallowConvNet
def square(x):
    return K.square(x)

def log(x):
    return K.log(K.clip(x, min_value = 1e-7, max_value = 10000)) 

In [ ]:

def ShallowConvNet(nb_classes, Chans = 64, Samples = 128, dropoutRate = 0.5):
    """ Keras implementation of the Shallow Convolutional Network as described
    in Schirrmeister et. al. (2017), Human Brain Mapping.
    
    Assumes the input is a 2-second EEG signal sampled at 128Hz. Note that in 
    the original paper, they do temporal convolutions of length 25 for EEG
    data sampled at 250Hz. We instead use length 13 since the sampling rate is 
    roughly half of the 250Hz which the paper used. The pool_size and stride
    in later layers is also approximately half of what is used in the paper.
    
    Note that we use the max_norm constraint on all convolutional layers, as 
    well as the classification layer. We also change the defaults for the
    BatchNormalization layer. We used this based on a personal communication 
    with the original authors.
    
                     ours        original paper
    pool_size        1, 35       1, 75
    strides          1, 7        1, 15
    conv filters     1, 13       1, 25    
    
    Note that this implementation has not been verified by the original 
    authors. We do note that this implementation reproduces the results in the
    original paper with minor deviations. 
    """

    # start the model
    input_main   = Input((Chans, Samples, 1))
    block1       = Conv2D(40, (1, 13), 
                                 input_shape=(Chans, Samples, 1),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(input_main)
    block1       = Conv2D(40, (Chans, 1), use_bias=False, 
                          kernel_constraint = max_norm(2., axis=(0,1,2)))(block1)
    block1       = BatchNormalization(epsilon=1e-05, momentum=0.1)(block1)
    block1       = Activation(square)(block1)
    block1       = AveragePooling2D(pool_size=(1, 35), strides=(1, 7))(block1)
    block1       = Activation(log)(block1)
    block1       = Dropout(dropoutRate)(block1)
    flatten      = Flatten()(block1)
    dense        = Dense(nb_classes, kernel_constraint = max_norm(0.5))(flatten)
    softmax      = Activation('softmax')(dense)
    
    return Model(inputs=input_main, outputs=softmax)

### DNC

In [ ]:
from tensorflow.keras.layers import Dense,Flatten,  Conv2D,BatchNormalization, AveragePooling2D, Dropout
from tensorflow.keras.layers import SpatialDropout2D, Reshape
from tensorflow.keras.optimizers import  Adamax
from tensorflow.keras.regularizers import l2, l1, l1_l2
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.layers import Activation


def gen_model_DNC( ): 
  
    inp = tf.keras.Input(shape=(features,Channels,1))
    c1 =Conv2D(30, (13, 1),kernel_regularizer=l2(0.001), kernel_constraint = max_norm(2., axis=(0,1,2)))(inp)
    c2=Conv2D(30, (1, Channels ), use_bias=False, kernel_regularizer=l1_l2(l1=1e-5, l2=0.001), kernel_constraint = max_norm(2., axis=(0,1,2)))(c1)
    bn=BatchNormalization(epsilon=1e-05, momentum=0.1)(c2)
    ac1=Activation(square)(bn)
    avg1= AveragePooling2D(pool_size=(100,1) , strides=(100, 1)  )(ac1)
    ac2= Activation(log)(avg1)
    sdout= SpatialDropout2D( 0.15 )(ac2)

    shallowConvNet_dim1=9
    shallowConvNet_dim2=30
    rs=Reshape(( shallowConvNet_dim1 , shallowConvNet_dim2 ))(sdout)

#     ----------------------------------  
    x = tf.keras.Input(shape=(None, 30, ))
    dnc_cell = DNC( 
      30,
      controller_units=64,
      memory_size=18,
      word_size=8,
      num_read_heads=2
    )
    dnc_initial_state = dnc_cell.get_initial_state(batch_size=512)
    rnn = tf.keras.layers.RNN(dnc_cell, return_sequences=True)
    rnn._name ='DNC_layer'
    y = rnn( rs, initial_state=dnc_initial_state)
#     ----------------------------------


    fl=Flatten ()(y)
    fc1 = Dense( 16 )(fl)
    do = Dropout(0.3 )(fc1)
    fc2 = Dense(2,activation='sigmoid')(do)
    model55 = tf.keras.models.Model(inputs=inp, outputs= fc2)
    
    

    opt=Adamax(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-5)     
    model55.compile(loss='binary_crossentropy', optimizer=opt,  metrics=['binary_accuracy'  , precision_m, recall_m, f1_m, sensitivity_m, specificity_m  ])
    return model55

b = gen_model_DNC()
b.summary()

### Model_Maker

In [10]:
def Model_Maker( ):  
  

  return gen_model_DNC() 

## Pipeline


In [11]:
## Pipeline library
from imblearn.pipeline import Pipeline, make_pipeline
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import StandardScaler

   
def make_pipeline():    

    main_pipeline = Pipeline(steps =[
        ('scaler',StandardScaler() ),           
        ('Reshaping' , CustomLSTMReshaper()),         
        ('classifier', KerasClassifier(  build_fn=Model_Maker  , batch_size=512, verbose=1, epochs=700    ))       
    ],  verbose=True )
    
    return main_pipeline

## Visualization

In [12]:
from plot_keras_history import plot_history
import matplotlib.pyplot as plt

def Visualize_Result(mp , k, path ,  dbType): 

  #pip install plot_keras_history
  #https://pypi.org/project/plot-keras-history/

  estimator= mp['classifier'] 
  history=estimator.model.history.history


  # plot_history(history)
  # plt.show()

  plot_history(history, path= path + dbType +"_"+ "_fold("+ str(k) +").png" )
  plt.close()
  return 


# post_Visualization

In [21]:

from matplotlib import pyplot as pl
import numpy as np



def dict_mean(dict_list):
  mean_dict = {}
  std_dict = {}
  

  for key in dict_list[0].keys():
      mean_dict[key] = np.mean([d[key] for d in dict_list], axis=0)
      std_dict[key] = np.std([d[key] for d in dict_list], axis=0)
  
  return mean_dict, std_dict


j=0 

def Auto_All_History_plot(all_history, savepath , dbType):
  #print(  all_history  )
  result=dict_mean(all_history )
  
  print()
  for val_metric_name in result[0]:
    #print('key', val_metric_name )
    
    if str( val_metric_name[0:4]) !='val_': # This is a condition for not calculating the results for both train and validation categories
        #break                               # Calculations are performed for train category and evaluation is also considered for validation
        continue
        
         
    metric_name=str( val_metric_name[4:])


    mean=result[0][ metric_name ] # mean
    std=result[1][ metric_name ] #std


    val_mean=result[0][ val_metric_name ] # mean
    val_std=result[1][ val_metric_name ] #std


    pl.plot(mean ,color='blue' , marker="x", markersize=10 ,markevery=70  )  
    pl.plot(val_mean ,color='red'  ,linestyle='dashed', markersize=20 )

    pl.fill_between(  range(len(mean)) , mean-std, mean + std , alpha=0.3, label='Train' ,color='blue' )
    pl.fill_between(  range(len(val_mean)) , val_mean-val_std, val_mean + val_std , alpha=0.3 , label='Validation' ,color='red' )

    if str( metric_name[-2:] )=='_m':
        metric_name=metric_name[:-2]
    

    plt.title( 'The Model '+metric_name+' Plot')
    plt.xlabel('Epoch')
    plt.ylabel( metric_name )

    plt.legend(loc='best')
    #pl.show()
    plt.savefig( savepath+dbType+ '_'+ metric_name+'.png')
    plt.close()

  return
Auto_All_History_plot( all_history , basePath ,dbType)


# Test Model

In [13]:
# test Unseen data
import pickle as pkl

def LoadUnseenData():
  
  with open(basePath+'Imagery_temp_X_test_IV2A.pkl','rb') as f: #read
    X_test = pkl.load(f)
    
  #print(X_test.shape)
  

  with open(basePath+'Imagery_temp_y_test_IV2A.pkl','rb') as f: #read
    Y_test = pkl.load(f)
    
    
    
    Y_test=[np.where(r==1)[0][0] for r in Y_test]
    
  #print(Y_test.shape)
  print('-------------------------------------------')

  return X_test, Y_test

ERROR! Session/line number was not unique in database. History logging moved to new session 123


# over_all_metrics

In [15]:
def over_all_metrics_stat(skf_metrics, savePath):
  # Take mean and std over all folds 
  average_metrics_all_folds = dict(pd.DataFrame(skf_metrics).mean())
  std_metrics_all_folds = dict(pd.DataFrame(skf_metrics).std())

  

  average_metrics_all_folds = {str( k[5:])+"_avg_folds":v for k,v in average_metrics_all_folds.items()}
  std_metrics_all_folds = {str( k[5:])+"_std_folds":v for k,v in std_metrics_all_folds.items()}

  print('_________ Training metric for all k-fold _________')
  print('average_metrics_all_folds')
  print(average_metrics_all_folds)
  print()
  print('std_metrics_all_folds')    
  print(std_metrics_all_folds)
  print()


  df_average_metrics_all_folds = pd.DataFrame([average_metrics_all_folds])
  df_std_metrics_all_folds = pd.DataFrame([std_metrics_all_folds])

  pd.concat([df_average_metrics_all_folds, df_std_metrics_all_folds ],axis=1 ).to_csv( savePath + dbType + '_UnSeen_data_metric_avg_std.csv')


  return     

    

# k-fold Writer

In [ ]:
from sklearn.model_selection import StratifiedKFold

n_splits=5
skf = StratifiedKFold(n_splits=n_splits, shuffle=False ) #,random_state=random_seed


    


folds = {}
count = 1
for train_index, val_index in skf.split(X_train, y_train2 ):
    folds['fold_{}'.format(count)] = {}
    folds['fold_{}'.format(count)]['train_index'] = train_index.tolist()
    folds['fold_{}'.format(count)]['val_index'] = val_index.tolist()
    count += 1
print(len(folds) == n_splits)#assert we have the same number of splits





In [ ]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ RUN ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
import pandas as pd
import gc
from keras.utils.np_utils import to_categorical 
import matplotlib.pyplot as plt
import pickle




i=1
 
for key, val in folds.items():


  print(key)
  print(i)

  train_index = val['train_index']
  val_index = val['val_index']


  
  skf_X_train, skf_X_val = (pd.DataFrame( X_train )).iloc[train_index], (pd.DataFrame( X_train)).iloc[val_index]
  skf_y_train, skf_y_val = (pd.DataFrame( y_train2 )).iloc[train_index], (pd.DataFrame( y_train2 )).iloc[val_index]



 
  with open( basePath + 'kfold_data_variable ('+ str(i) +').pkl', 'wb') as p: 
      pickle.dump([ skf_X_train, skf_X_val, skf_y_train, skf_y_val  ], p)


  print('_step_',i)
  i+=1
  

# k-fold Reader

In [16]:
# load previous config
import pickle as pkl
import pandas as pd


# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
try:
  with open(basePath + dbType +'_Kfold_Imagery_all_history.pkl','rb') as f: #read
    all_history = pkl.load(f)

  with open(basePath + dbType+ '_UnSeen_data_metric_avg_std.pkl','rb') as f: #read
    skf_metrics = pkl.load(f)


except:
  print('var empty mod')
  all_history= []
  skf_metrics= []
 
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# Main Excutation

In [22]:
print(len(all_history))

5


In [ ]:

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ RUN ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
import pandas as pd
import gc
from keras.utils.np_utils import to_categorical 
import matplotlib.pyplot as plt
import pickle 


gc.collect()


i=5


with open( basePath + 'kfold_data_variable ('+ str( i) +').pkl' ,'rb') as p:   
    skf_X_train, skf_X_val, skf_y_train, skf_y_val  = pickle.load(p)

gc.collect()


print("============ Training on fold ({}) ==============".format(i))

#skf_y_val2=[np.where(r==1)[0][0] for r in np.array(skf_y_val) ]


skf_y_train2= to_categorical( skf_y_train , num_classes=2)
skf_y_val2= to_categorical( skf_y_val , num_classes=2)




pipeline = make_pipeline( ) 

# ------------------------
pipeline.named_steps['scaler'].fit(skf_X_train)
skf_X_val22=pipeline.named_steps['scaler'].transform(skf_X_val)
skf_X_val2=pipeline.named_steps['Reshaping'].transform(skf_X_val22)
# ------------------------





pipeline.fit(skf_X_train, skf_y_train2 , classifier__validation_data=( skf_X_val2, skf_y_val2)  )

gc.collect()
del skf_X_train, skf_y_train, skf_y_val, skf_y_train2, skf_y_val2, skf_X_val2, skf_X_val22, skf_X_val







X_test, Y_test =LoadUnseenData()

y_pred_val = pipeline.predict( X_test  )
del X_test
gc.collect()

print("_________________________________________________")
print()





print('___step___',i)
print()
print('Model Evaluation is in process...')
test_results = get_experiments( Y_test , y_pred_val)
print(test_results)
del Y_test , y_pred_val

gc.collect()

skf_metrics.append(test_results)



# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
all_history.append( pipeline['classifier'].model.history.history )
print('Model Training history has been store')
print()
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


Visualize_Result(pipeline , i, basePath ,   dbType )
print('Evaluation Resault has been Visualized')
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


# # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
del pipeline
gc.collect()

# # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


with open(basePath + dbType +'_Kfold_Imagery_all_history.pkl','wb') as f: #write
  pkl.dump( all_history , f)
print('Model Training history Resault has been wrote on file')


with open(basePath + dbType +'_UnSeen_data_metric_avg_std.pkl','wb') as f: #write
  pkl.dump( skf_metrics , f)
print('Model Evaliuation Resault has been wrote on file')  


print()


In [ ]:
print(len(all_history))
#all_history.append( pipeline['classifier'].model.history.history )

In [ ]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# get results
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# pd.DataFrame([skf_metrics]).to_csv( basePath+ dbType + '_All_fold_Results_Metric.csv')

over_all_metrics_stat(skf_metrics, basePath)


# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
pd.DataFrame([all_history]).to_csv( basePath+ dbType+'_All_history.csv')

Auto_All_History_plot( all_history , basePath ,dbType)

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# pd.DataFrame([all_UnseenData_results]).to_csv( basePath+ dbType + '_All_UnseenData_Results.csv')

df = pd.DataFrame( skf_metrics, columns=['test_accuracy', 'test_precision', 'test_recall' ,'test_f1' , 'test_specificity', 'test_sensitivity', 'test_kappa' ])
pd.DataFrame( df ).to_csv( basePath + dbType + '_All_UnseenData_Results_df.csv' , index=False )

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


print(' ok ')

